In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point, LineString
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import Image
import selenium.webdriver
import shapely.geometry
import osmnx as ox
import pyproj
from geojson.feature import Feature, FeatureCollection
from shapely.geometry import Point, Polygon, LineString
import h3
import json
import branca.colormap as cmp
import glob
from statistics import *

In [2]:
import os
cwd = os.getcwd()
os.chdir(os.path.join(cwd,'..\..\..'))
import ExMAS.main
import ExMAS.utils

from ExMAS.utils import inData as inData

In [3]:
def ride_finder(dem, disc):
    dem = "_"+str(dem)+"_"
    disc= "_"+str(disc).replace(".","")+"_"
    rides = glob.glob('ExMAS/data/results/rides/*.csv', recursive=True)
    rides  = [s for s in rides if dem in s]
    rides = [s for s in rides if disc in s]
    return rides[0]

def req_finder(dem, disc):
    dem = "_"+str(dem)+"_"
    disc= "_"+str(disc).replace(".","")+"_"
    req = glob.glob('ExMAS/data/results/requests/*.csv', recursive=True)
    req  = [s for s in req if dem in s]
    req = [s for s in req if disc in s]
    return req[0]
    

In [4]:
def plot_hex_map(inData,res,dem,disc, APERTURE_SIZE = 8, threshold = 1, col = 'u_diff'):
    res['hex_o_{}'.format(APERTURE_SIZE)] = res.apply(lambda row: h3.geo_to_h3(row.y,row.x,APERTURE_SIZE),axis = 1) 
    trips = res
    
    col_geom = 'hex_o_{}'.format(APERTURE_SIZE)
    hexes = pd.Series(list(set(list(trips[col_geom].unique())+list(trips[col_geom].unique())))).to_frame(col_geom)
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['nobs'] = trips.groupby(col_geom).size()
    hexes = hexes[hexes['nobs']>threshold]
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)
    aggr = trips.groupby(col_geom)[col]
    hexes['nobs'] = aggr.size()
    hexes[col] = aggr.mean()
    hexes[col+'_std'] = aggr.std()
    list_features = []
    for i, row in hexes.iterrows():
        feature = Feature(geometry = row["geom"],
                          id = row[col_geom],
                          properties = {"resolution": 9})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    CENTER = list(inData.nodes.loc[inData.stats.center][['y','x']].values)
    tile = 'cartodbpositron'
    linear = cmp.LinearColormap(['red', 'yellow', 'green'],vmin=min(res[col]), 
                                vmax=max(res[col]),caption='Color Scale for Map') #Caption for Color scale or Legend
    base_map = folium.Map(location=CENTER, zoom_start=12,tiles=tile, zoomControl =  False)
    bins = [-0.1,0,0.0025*max(res[col]),0.005*max(res[col]),0.01*max(res[col]),0.05*max(res[col])]
            
    m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color='BuPu', control = False,
            fill_opacity=0.7, line_opacity=0.1,legend_name=col+" for demand {} at discount {}".format(dem,disc)).add_to(base_map)
    '''m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color=linear, control = False,
            fill_opacity=0.7, line_opacity=0.1,legend_name=col).add_to(base_map)'''
    m.save('map.html')
    return base_map
    

In [5]:
params = ExMAS.utils.get_config('ExMAS/data/configs/my_config.json') # load the default 

In [ ]:
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph

In [ ]:
inData = ExMAS.utils.load_albatross_csv(inData, params, sample=True)

In [ ]:
demand = 3000
discount   = 0.15
ride_finder(demand,discount)

In [ ]:
df_ride = pd.read_csv(ride_finder(demand,discount))
df_req  = pd.read_csv(req_finder(demand,discount))

In [ ]:
resop = pd.DataFrame(columns=['lambda_r','u_veh','u_veh_ns', 'index_origin','index_dest','origin','destination'])
resop = resop.fillna(0)



for i in range(len(df_ride)):
    a = df_ride['lambda_r'].iloc[i]
    b = json.loads(df_ride['indexes_orig'].iloc[i])
    c = df_req['origin']
    d = df_req['destination']
    e = df_ride['u_veh'].iloc[i]
    f = df_ride['PassHourTrav_ns'].iloc[i]
    g = json.loads(df_ride['indexes_dest'].iloc[i])
    h = df_ride['kind'].iloc[i]
    if 20<=h<30:
        h=2
    elif 30<=h<40:
        h=3
    elif 40<=h<50:
        h=4
    elif h>=50:
        h=5
    else:
        h=1
    for j in range(len(b)):
        resop = resop.append({'lambda_r':a,'u_veh':e,'u_veh_ns':f,'index_origin':b[j],'index_dest':g[j],\
                              'origin':c[b[j]],'destination':d[g[j]],'x':c[b[j]],'y':c[b[j]],'degree':h},ignore_index=True)
        
resop['lam_diff'] = resop['lambda_r']
resop['u_diff']   = resop['u_veh']-resop['u_veh_ns']
resop['x'] = resop['x'].apply(lambda row: inData.nodes.loc[row].x)
resop['y'] = resop['y'].apply(lambda row: inData.nodes.loc[row].y)


resop_s = resop[resop['degree']>1]           #filter only sharing rides

In [ ]:
mean(resop['degree'])

In [ ]:
linear = cmp.LinearColormap(['red', 'yellow', 'green'],vmin=min(resop['lambda_r']), vmax=max(resop['lambda_r']),caption='Color Scale for Map')
#style_function=lambda feature: 
 #                           'fillColor': linear(resop[feature['lambda_r']]),
colour = resop['lambda_r'].apply(lambda x: linear(x))

# Discount = 0.15

In [ ]:
plot_hex_map(inData, resop_s,demand,discount, APERTURE_SIZE = 8, col = 'lambda_r')

# Discount = 0.2

In [ ]:
discount   = 0.2
ride_finder(demand,discount)

In [ ]:
df_ride = pd.read_csv(ride_finder(demand,discount))
df_req  = pd.read_csv(req_finder(demand,discount))

In [ ]:
resop = pd.DataFrame(columns=['lambda_r','u_veh','u_veh_ns', 'index_origin','index_dest','origin','destination'])
resop = resop.fillna(0)



for i in range(len(df_ride)):
    a = df_ride['lambda_r'].iloc[i]
    b = json.loads(df_ride['indexes_orig'].iloc[i])
    c = df_req['origin']
    d = df_req['destination']
    e = df_ride['u_veh'].iloc[i]
    f = df_ride['PassHourTrav_ns'].iloc[i]
    g = json.loads(df_ride['indexes_dest'].iloc[i])
    h = df_ride['kind'].iloc[i]
    if 20<=h<30:
        h=2
    elif 30<=h<40:
        h=3
    elif 40<=h<50:
        h=4
    elif h>=50:
        h=5
    else:
        h=1
    for j in range(len(b)):
        resop = resop.append({'lambda_r':a,'u_veh':e,'u_veh_ns':f,'index_origin':b[j],'index_dest':g[j],\
                              'origin':c[b[j]],'destination':d[g[j]],'x':c[b[j]],'y':c[b[j]],'degree':h},ignore_index=True)
        
resop['lam_diff'] = resop['lambda_r']
resop['u_diff']   = resop['u_veh']-resop['u_veh_ns']
resop['x'] = resop['x'].apply(lambda row: inData.nodes.loc[row].x)
resop['y'] = resop['y'].apply(lambda row: inData.nodes.loc[row].y)


resop_s = resop[resop['degree']>1]           #filter only sharing rides

In [ ]:
plot_hex_map(inData, resop_s,demand,discount, APERTURE_SIZE = 8, col = 'lambda_r')

# Discount = 0.25

In [ ]:
discount   = 0.25
ride_finder(demand,discount)

In [ ]:
df_ride = pd.read_csv(ride_finder(demand,discount))
df_req  = pd.read_csv(req_finder(demand,discount))

In [ ]:
resop = pd.DataFrame(columns=['lambda_r','u_veh','u_veh_ns', 'index_origin','index_dest','origin','destination'])
resop = resop.fillna(0)



for i in range(len(df_ride)):
    a = df_ride['lambda_r'].iloc[i]
    b = json.loads(df_ride['indexes_orig'].iloc[i])
    c = df_req['origin']
    d = df_req['destination']
    e = df_ride['u_veh'].iloc[i]
    f = df_ride['PassHourTrav_ns'].iloc[i]
    g = json.loads(df_ride['indexes_dest'].iloc[i])
    h = df_ride['kind'].iloc[i]
    if 20<=h<30:
        h=2
    elif 30<=h<40:
        h=3
    elif 40<=h<50:
        h=4
    elif h>=50:
        h=5
    else:
        h=1
    for j in range(len(b)):
        resop = resop.append({'lambda_r':a,'u_veh':e,'u_veh_ns':f,'index_origin':b[j],'index_dest':g[j],\
                              'origin':c[b[j]],'destination':d[g[j]],'x':c[b[j]],'y':c[b[j]],'degree':h},ignore_index=True)
        
resop['lam_diff'] = resop['lambda_r']
resop['u_diff']   = resop['u_veh']-resop['u_veh_ns']
resop['x'] = resop['x'].apply(lambda row: inData.nodes.loc[row].x)
resop['y'] = resop['y'].apply(lambda row: inData.nodes.loc[row].y)


resop_s = resop[resop['degree']>1]           #filter only sharing rides

In [ ]:
plot_hex_map(inData, resop_s,demand,discount, APERTURE_SIZE = 8, col = 'lambda_r')

# Discount = 0.3

In [ ]:
discount   = 0.3
ride_finder(demand,discount)

In [ ]:
df_ride = pd.read_csv(ride_finder(demand,discount))
df_req  = pd.read_csv(req_finder(demand,discount))

In [ ]:
resop = pd.DataFrame(columns=['lambda_r','u_veh','u_veh_ns', 'index_origin','index_dest','origin','destination'])
resop = resop.fillna(0)



for i in range(len(df_ride)):
    a = df_ride['lambda_r'].iloc[i]
    b = json.loads(df_ride['indexes_orig'].iloc[i])
    c = df_req['origin']
    d = df_req['destination']
    e = df_ride['u_veh'].iloc[i]
    f = df_ride['PassHourTrav_ns'].iloc[i]
    g = json.loads(df_ride['indexes_dest'].iloc[i])
    h = df_ride['kind'].iloc[i]
    if 20<=h<30:
        h=2
    elif 30<=h<40:
        h=3
    elif 40<=h<50:
        h=4
    elif h>=50:
        h=5
    else:
        h=1
    for j in range(len(b)):
        resop = resop.append({'lambda_r':a,'u_veh':e,'u_veh_ns':f,'index_origin':b[j],'index_dest':g[j],\
                              'origin':c[b[j]],'destination':d[g[j]],'x':c[b[j]],'y':c[b[j]],'degree':h},ignore_index=True)
        
resop['lam_diff'] = resop['lambda_r']
resop['u_diff']   = resop['u_veh']-resop['u_veh_ns']
resop['x'] = resop['x'].apply(lambda row: inData.nodes.loc[row].x)
resop['y'] = resop['y'].apply(lambda row: inData.nodes.loc[row].y)


resop_s = resop[resop['degree']>1]           #filter only sharing rides

In [ ]:
plot_hex_map(inData, resop_s,demand,discount, APERTURE_SIZE = 8, col = 'lambda_r)

# Discount = 0.35

In [ ]:
discount   = 0.35
ride_finder(demand,discount)

In [ ]:
df_ride = pd.read_csv(ride_finder(demand,discount))
df_req  = pd.read_csv(req_finder(demand,discount))

In [ ]:
resop = pd.DataFrame(columns=['lambda_r','u_veh','u_veh_ns', 'index_origin','index_dest','origin','destination'])
resop = resop.fillna(0)



for i in range(len(df_ride)):
    a = df_ride['lambda_r'].iloc[i]
    b = json.loads(df_ride['indexes_orig'].iloc[i])
    c = df_req['origin']
    d = df_req['destination']
    e = df_ride['u_veh'].iloc[i]
    f = df_ride['PassHourTrav_ns'].iloc[i]
    g = json.loads(df_ride['indexes_dest'].iloc[i])
    h = df_ride['kind'].iloc[i]
    if 20<=h<30:
        h=2
    elif 30<=h<40:
        h=3
    elif 40<=h<50:
        h=4
    elif h>=50:
        h=5
    else:
        h=1
    for j in range(len(b)):
        resop = resop.append({'lambda_r':a,'u_veh':e,'u_veh_ns':f,'index_origin':b[j],'index_dest':g[j],\
                              'origin':c[b[j]],'destination':d[g[j]],'x':c[b[j]],'y':c[b[j]],'degree':h},ignore_index=True)
        
resop['lam_diff'] = resop['lambda_r']
resop['u_diff']   = resop['u_veh']-resop['u_veh_ns']
resop['x'] = resop['x'].apply(lambda row: inData.nodes.loc[row].x)
resop['y'] = resop['y'].apply(lambda row: inData.nodes.loc[row].y)


resop_s = resop[resop['degree']>1]           #filter only sharing rides

In [ ]:
plot_hex_map(inData, resop_s,demand,discount, APERTURE_SIZE = 8, col = 'lambda_r')